In [24]:
import spacy
import pandas as pd 
import numpy as np 
from spacy import nlp, DocBin

ImportError: cannot import name 'nlp' from 'spacy' (/home/alwallace/miniconda3/envs/spacy/lib/python3.8/site-packages/spacy/__init__.py)

In [ ]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode_escape')

In [ ]:
df.head(50)

In [ ]:
start_idx = list(np.ravel(np.where(~pd.isnull(df['Sentence #']))))

In [ ]:
start_idx

In [45]:
js = [None] * (len(start_idx)-1)
for (b,e),i in zip(zip(start_idx[:-1],[x-1 for x in start_idx[1:]]),range(0, (len(start_idx)-1))):
    sent = list(df['Word'][b:e])
    tags = [None if t=='O' else t.split('-')[1] for t in list(df['Tag'][b:e])]
    ents = []
    w_start = 0
    ent = [None,None,None]
    for w_idx in range(0,len(sent)):
        word_len = len(sent[w_idx])
        same_tag = False
        tag = tags[w_idx]
        try:
            if tags[w_idx+1] == tag and tag is not None:
                same_tag = True
        except:
            pass 
        if same_tag:
            if ent[0] is None:
                ent[0] = tag
                ent[1] = w_start
            else:
                pass
        else:
            if ent[0] is None:
                ent[0] = tag
                ent[1] = w_start
                ent[2] = w_start + word_len
            else:
                ent[2] = w_start + word_len
            if ent[0] is not None:
                ents.append((tuple(ent)))
            ent = [None,None,None]
        w_start = w_start + word_len + 1
    js[i] = {'sentence' : ' '.join(sent), 
    'entities' : ents}
    
    

In [46]:
js[30000]

{'sentence': "Hamas wants to build a coalition government , but Mr. Abbas 's Fatah party and the militant Islamic Jihad group have declined to join",
 'entities': [('org', 0, 5),
  ('per', 50, 59),
  ('geo', 63, 68),
  ('geo', 92, 105)]}

In [48]:
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en")
# the DocBin will store the example documents
db = DocBin()
for dic in js:
    text = dic['sentence']
    annotations = dic['entities']
    doc = nlp(text)
    ents = []
    for label, start, end in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./train.spacy")